<a href="https://colab.research.google.com/github/AnkitaSK/lab-sql-query-from-table-names-continued/blob/main/lab-sql-query-from-table-names-continued.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL query from table names - Continued

In [14]:
from openai import OpenAI
import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

## The old Prompt

In [15]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [16]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [17]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [20]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [21]:
#new
context_user = context.copy()
print(return_CCRMSQL("Highest paid salary", context_user))

```sql
SELECT MAX(salary) AS highest_paid_salary
FROM employees;
```


In [22]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Highest paid salary", old_context_user))

This is your SQL:
```sql
SELECT MAX(salary) AS highest_paid_salary
FROM salary;
```

This SQL query selects the maximum (highest) salary from the "salary" table, returning it as "highest_paid_salary".


In [23]:
#new
print(return_CCRMSQL("Return the Institution with a higher average salary", context_user))

```sql
SELECT Institution, AVG(Salary) AS AverageSalary
FROM Salaries
GROUP BY Institution
ORDER BY AverageSalary DESC
LIMIT 1;
```


In [24]:
#old
print(return_CCRMSQL("Return the Institution with a higher average salary", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` on the user ID to calculate the average salary for each institution. It then selects the institution with the highest average salary by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?